# Select M dwarfs through heuristics

In [1]:
import matplotlib.pyplot as plt
import numpy as np

In [2]:
import pandas as pd

In [3]:
from astropy.io import fits

In [4]:
hdus = fits.open('../../data/Gaia/kepler_dr3_good.fits')

### Read in the table to a pandas dataframe

In [5]:
from astropy.table import Table

In [6]:
tab = Table()

In [7]:
for col in hdus[1].columns:
    tab[col.name] = hdus[1].data[col.name]

In [9]:
df = tab.to_pandas()

In [15]:
df.tail()

,kepid,ra_kic,dec_kic,source_id,random_index,ra,dec,parallax,parallax_error,parallax_over_error,...,mass_err2,prov_sec,nconfp,nkoi,ntce,jmag,hmag,kmag,planet?,kepler_gaia_mag_diff
196757,12984227,290.07986,52.453712,2139330189463960320,1434812375,290.079838,52.453692,0.246011,0.018616,13.214761,...,-0.484,DSEP,0,0,0,8.505,7.589,7.295,none,-0.332914
196758,12984288,290.11249,52.415588,2139329875927860736,662936371,290.112599,52.415592,0.005198,0.055361,0.093884,...,-0.157,DSEP,0,0,0,16.289,15.603,15.306,none,-0.235144
196759,12984307,290.12332,52.474621,2139330636140575616,923414311,290.123462,52.474735,1.779882,0.009830,181.061508,...,-0.375,DSEP,0,0,0,11.459,11.104,11.075,none,-0.035185
196760,12984404,290.17230,52.423828,2139329467909467008,595517067,290.172325,52.423856,0.707362,0.009949,71.101357,...,-2.250,DSEP,0,0,0,9.851,9.212,9.095,none,-0.027551
196761,12984422,290.18420,52.404362,2139329399189739392,1570613422,290.184199,52.404460,0.975763,0.024997,39.035137,...,-0.102,DSEP,0,0,0,14.314,13.814,13.747,none,-0.040604


### Criterion 1: Teff and logg cuts

In [12]:
criterion1 = df.teff < 3400

In [13]:
criterion1.sum()

989

### Criterion 2: